In [2]:
import numpy as np
import pickle
from collections import defaultdict
import random

class TicTacToe:
    def __init__(self):
        self.board = [" " for _ in range(9)]
        self.current_winner = None

    def print_board(self):
        for row in [self.board[i*3:(i+1)*3] for i in range(3)]:
            print("| " + " | ".join(row) + " |")

    def available_moves(self):
        return [i for i, spot in enumerate(self.board) if spot == " "]

    def empty_squares(self):
        return " " in self.board

    def num_empty_squares(self):
        return self.board.count(" ")

    def make_move(self, square, letter):
        if self.board[square] == " ":
            self.board[square] = letter
            if self.winner(square, letter):
                self.current_winner = letter
            return True
        return False

    def winner(self, square, letter):
        # Check row
        row_ind = square // 3
        row = self.board[row_ind*3: (row_ind + 1)*3]
        if all([spot == letter for spot in row]):
            return True

        # Check column
        col_ind = square % 3
        column = [self.board[col_ind+i*3] for i in range(3)]
        if all([spot == letter for spot in column]):
            return True

        # Check diagonal
        if square % 2 == 0:
            diagonal1 = [self.board[i] for i in [0, 4, 8]]
            if all([spot == letter for spot in diagonal1]):
                return True
            diagonal2 = [self.board[i] for i in [2, 4, 6]]
            if all([spot == letter for spot in diagonal2]):
                return True

        return False

class QLearningAgent:
    def __init__(self, epsilon=0.1, alpha=0.5, gamma=0.9):
        self.q_table = defaultdict(lambda: defaultdict(lambda: 0))
        self.epsilon = epsilon  # exploration rate
        self.alpha = alpha     # learning rate
        self.gamma = gamma     # discount factor

    def get_state_key(self, board):
        return ''.join(board)

    def choose_action(self, game, letter):
        state = self.get_state_key(game.board)
        if random.random() < self.epsilon:
            return random.choice(game.available_moves())

        return self.get_best_action(game, state)

    def get_best_action(self, game, state):
        available_moves = game.available_moves()
        if not available_moves:
            return None

        # Get Q-values for all available actions
        q_values = {action: self.q_table[state][action] for action in available_moves}
        max_q = max(q_values.values())

        # Get all actions that have the maximum Q-value
        best_actions = [action for action, q_value in q_values.items() if q_value == max_q]
        return random.choice(best_actions)

    def learn(self, state, action, reward, next_state):
        best_next_action = max(self.q_table[next_state].values()) if self.q_table[next_state] else 0
        current_q = self.q_table[state][action]
        new_q = current_q + self.alpha * (reward + self.gamma * best_next_action - current_q)
        self.q_table[state][action] = new_q

def train(n_games=10000):
    agent = QLearningAgent()

    for _ in range(n_games):
        game = TicTacToe()
        current_player = 'X'

        while game.empty_squares():
            if current_player == 'X':
                # Agent's turn
                state = agent.get_state_key(game.board)
                action = agent.choose_action(game, current_player)

                # Make move
                game.make_move(action, current_player)

                # Get reward
                if game.current_winner == 'X':
                    reward = 1
                elif game.num_empty_squares() == 0:
                    reward = 0.5  # Draw
                else:
                    reward = 0

                # Get new state
                new_state = agent.get_state_key(game.board)

                # Learn from this move
                agent.learn(state, action, reward, new_state)

                if game.current_winner:
                    break
            else:
                # Random opponent
                square = random.choice(game.available_moves())
                game.make_move(square, current_player)
                if game.current_winner:
                    break

            current_player = 'O' if current_player == 'X' else 'X'

    return agent

def play_against_agent(agent):
    game = TicTacToe()
    current_player = 'X'  # X will be the agent

    while game.empty_squares():
        if current_player == 'X':
            # Agent's turn
            square = agent.choose_action(game, current_player)
            print(f"\nAgent chose position {square}")
        else:
            # Human's turn
            game.print_board()
            valid_square = False
            while not valid_square:
                square = input(f"\nYour turn! Input move (0-8): ")
                try:
                    square = int(square)
                    if square not in game.available_moves():
                        raise ValueError
                    valid_square = True
                except ValueError:
                    print("Invalid square. Try again.")

        game.make_move(square, current_player)

        if game.current_winner:
            game.print_board()
            print(f"\n{current_player} wins!")
            return

        current_player = 'O' if current_player == 'X' else 'X'

    game.print_board()
    print("\nIt's a tie!")

# Train the agent
print("Training the agent...")
trained_agent = train(10000)
print("Training complete!")

# Play against the trained agent
while True:
    play_against_agent(trained_agent)
    play_again = input("\nWould you like to play again? (y/n): ")
    if play_again.lower() != 'y':
        break

print("\nThanks for playing!")

Training the agent...
Training complete!

Agent chose position 6
|   |   |   |
|   |   |   |
| X |   |   |

Your turn! Input move (0-8): 6
Invalid square. Try again.

Your turn! Input move (0-8): 4

Agent chose position 7
|   |   |   |
|   | O |   |
| X | X |   |

Your turn! Input move (0-8): 8

Agent chose position 0
| X |   |   |
|   | O |   |
| X | X | O |

Your turn! Input move (0-8): 3

Agent chose position 5
| X |   |   |
| O | O | X |
| X | X | O |

Your turn! Input move (0-8): 2

Agent chose position 1
| X | X | O |
| O | O | X |
| X | X | O |

It's a tie!

Would you like to play again? (y/n): 0

Thanks for playing!
